# Scraping Data Kampus Merdeka MSIB: Magang Melalui API

In [ ]:
import   warnings
warnings.filterwarnings("ignore")

In [ ]:
import   os
import   glob
import   json
import   time
import   folium
import   logging
import   requests
import   itertools
import   branca.colormap as cm

In [ ]:
from     pyforest                          import *
from     math                              import sqrt
from     datetime                          import datetime
from     selenium                          import webdriver
from     bs4                               import BeautifulSoup, Tag
from     concurrent.futures                import ThreadPoolExecutor, as_completed
from     selenium.webdriver.support.select import Select

#### Karena jika menggunakan scraping lewat css terlalu susah kwkwkw. Jadi akses datanya lewat API kampus merdeka

In [3]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

mengatur basic konfigurasi terhadap logging untuk mencatat informasi dengan level INFO. Nah nanti si format logging akan menampilkan waktu, level log, dan pesan. Intinya buat mencatat kejadian-kejadian penting dalam program dengan lebih rapi dan mudah dibaca

In [4]:
base_url = 'https://api.kampusmerdeka.kemdikbud.go.id/magang/browse/opportunities'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36 Edg/125.0.0.0'
}

- 'base_url' : 'adalah link api untuk mengakses setiap data lowongan magang (tidak termasuk deskripsi)'
- 'headers'  : 'adalah agent untuk mengambil data

In [5]:
# bikin fungsi yang mendapatkan data dari API dengan backoff eksponensial
def get_data_from_api(url):
    
    max_retries      = 5
    backoff_factor   = 1
    
    # berarti 5 kali percobaan
    for i in range(max_retries):
        try:
            response = requests.get(url, headers=headers)
            
            # memeriksa apakah request berhasil?
            response.raise_for_status()
            
            # kalau berhasil return hasil json
            return response.json()
        
        # ini kalau requestnya gagal
        except requests.exceptions.HTTPError as http_err:
            
            # 429 itu artinya too many request yaa
            if response.status_code == 429:
                logging.warning(f"HTTP error 429: Too Many Requests. Retrying in {backoff_factor} seconds...")
                time.sleep(backoff_factor)
                
                # kalau masih '429' juga, maka backoff akan dipangkat 2
                backoff_factor *= 2
                
            else:
                logging.error(f"HTTP error occurred: {http_err}")
                return None
            
        except Exception as err:
            
            logging.error(f"Other error occurred: {err}")
            return None
        
    return None

- `max_retries`: maksimun berapa kali pengulangan ketika request gagal.
- `backoff_factor`: berapa detik pertama ketika request gagal lalu dicoba lagi

In [6]:
data = get_data_from_api(base_url)

In [7]:
# memeriksa apakah data dari api berhasil diambil, sekaligus mencetak datanya
if data:
    
    # menampung semua data dari `data`
    opportunities = data.get('data', [])

    # list arr kosong utk simpan data
    all_data = []

    # fungsi didalam fungsi, buat aambil data deskripsi setiap lowongan
    def fetch_detail(id_lowongan):
        
        detail_url = f'https://api.kampusmerdeka.kemdikbud.go.id/magang/browse/position/{id_lowongan}'
        return get_data_from_api(detail_url)

    # ThreadPoolExecutor untuk melakukan paralelisme
    with ThreadPoolExecutor(max_workers=5) as executor:  # Mengurangi jumlah paralelisme
        
        future_to_id = {executor.submit(fetch_detail, opportunity.get('id')): opportunity for opportunity in opportunities}

        # perulangan untuk mengumpulkan data di link api pertama 
        for future in as_completed(future_to_id):
            
            opportunity       = future_to_id[future]
            id_lowongan       = opportunity.get('id')
            durasi            = opportunity.get('months_duration')
            mitra             = opportunity.get('mitra_name')
            brand_mitra       = opportunity.get('mitra_brand_name')
            logo              = opportunity.get('logo')

            row_data = {
                'id_lowongan' : id_lowongan,
                'durasi'      : durasi,
                'mitra'       : mitra,
                'brand_mitra' : brand_mitra,
                'logo'        : logo,
            }

            # mengumpulkan data di link api kedua atau deskripsi dari lowongan
            try:
                detail_data   = future.result()
                
                # jika berhasil, maka memperbarui baris datanya
                if detail_data:
                    
                    detail    = detail_data.get('data', {}) #key dari json 'data'
                    row_data.update({
                        'nama_lowongan'    : detail.get('name'),
                        'detail_nama'      : detail.get('additional_title'),
                        'persyaratan'      : detail.get('requirement'),
                        'nama_aktifitas'   : detail.get('activity_id', {}).get('name'),
                        'deskripsi'        : detail.get('activity_id', {}).get('description'),
                        'detail_deskripsi' : detail.get('activity_id', {}).get('additional_information'),
                        'lokasi'           : detail.get('activity_id', {}).get('location'),
                        'pelaksanaan'      : detail.get('activity_id', {}).get('activity_type'),
                        'sertifikasi'      : detail.get('activity_id', {}).get('certified'),
                    })
                logging.info(f"Successfully fetched details for id {id_lowongan}")
                
            except Exception as exc:
                logging.error(f"Error fetching detail for id {id_lowongan}: {exc}")

            all_data.append(row_data)

            # delay
            time.sleep(1)

    # dataframe baru
    df = pd.DataFrame(all_data)

    print(df)
else:
    print("Gagal mendapatkan data dari API utama.")

Streaming output truncated to the last 5000 lines.
ERROR:root:HTTP error occurred: 400 Client Error: Bad Request for url: https://api.kampusmerdeka.kemdikbud.go.id/magang/browse/position/69223b99-36be-4363-a2bf-27fe6a005eb4
ERROR:root:HTTP error occurred: 400 Client Error: Bad Request for url: https://api.kampusmerdeka.kemdikbud.go.id/magang/browse/position/93e55449-9497-4f8c-aa1b-4eaa7961d716
ERROR:root:HTTP error occurred: 400 Client Error: Bad Request for url: https://api.kampusmerdeka.kemdikbud.go.id/magang/browse/position/65689240-8dfb-43c6-be9e-44c296112c02
ERROR:root:HTTP error occurred: 400 Client Error: Bad Request for url: https://api.kampusmerdeka.kemdikbud.go.id/magang/browse/position/9a7374e4-841e-4da2-a670-57527d237398
ERROR:root:HTTP error occurred: 400 Client Error: Bad Request for url: https://api.kampusmerdeka.kemdikbud.go.id/magang/browse/position/ce3678e6-acc0-411d-8600-95dedab1e24a
ERROR:root:HTTP error occurred: 400 Client Error: Bad Request for url: https://api.k

                               id_lowongan  durasi  \
0     60636725-0494-11ef-9e17-0a54edb93563       4   
1     1b162247-048d-11ef-bb03-ceeddaa1b367       4   
2     190c81c5-048a-11ef-9e17-0a54edb93563       4   
3     90665e71-0490-11ef-bb7f-6ac06bc5928b       4   
4     f2b8c752-04a0-11ef-8394-36f764739585       4   
...                                    ...     ...   
7411  039c521f-4ac2-46a0-b53b-33d487fee509       4   
7412  e3e2c034-0365-44b4-badb-c33a22fb33b7       3   
7413  45bf7d76-36a6-4496-b920-b2cfab9da4e3       4   
7414  3049aa1b-9420-4e01-a1dc-36982bd0c054       6   
7415  da0fa84e-d0eb-4419-8f4f-0480eb476b1b       4   

                                                  mitra       brand_mitra  \
0     Dinas Lingkungan Hidup dan Kehutanan Daerah Is...          DLHK DIY   
1     Dinas Lingkungan Hidup dan Kehutanan Daerah Is...          DLHK DIY   
2     Dinas Lingkungan Hidup dan Kehutanan Daerah Is...          DLHK DIY   
3     Dinas Lingkungan Hidup dan Kehutanan 

`ThreadPoolExecutor`: itu untuk mempercepat proses pengambilan detail posisi magang secara paralel dengan maksimal 5 thread. `executor.submit` mengirimkan tugas 'fetch_detail' untuk setiap `id` dalam `opportunities`, dan menyimpan future yang dihasilkan dalam dictionary `future_to_id`.

In [8]:
csv_path = r'C:\Users\ASUS\kode\Projek\15. msib_Rekomendasi\data_lowongan.csv'
df.to_csv(csv_path, index=False)